In [1]:
import keras
import keras.backend as K
import kapre
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
import random
from soph import soph_scaler, center_wave, ex_generator

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [43]:
input_layer = keras.layers.Input(shape=(1, 16000))
mel = kapre.time_frequency.Melspectrogram(
        sr=16000,
        n_mels=63,
        n_dft=512,
        power_melgram=2.0,
        trainable_kernel=False,
        return_decibel_melgram=True)(input_layer)
# mel = keras.layers.ZeroPadding2D(padding=((0,1), (0,1)))(mel)

In [44]:
x = keras.layers.Conv2D(128, (3, 3), activation='elu', padding='same')(mel)
x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same')(x)
x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same')(x)
encoded = keras.layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same')(encoded)
x = keras.layers.UpSampling2D((2, 2))(x)
x = keras.layers.Conv2D(64, (3, 3), activation='elu', padding='same')(x)
x = keras.layers.UpSampling2D((2, 2))(x)
x = keras.layers.Conv2D(128, (3, 3), activation='elu', padding='same')(x)
x = keras.layers.UpSampling2D((2, 2))(x)

x = keras.layers.Cropping2D(cropping=((1, 0), (1, 0)))(x)
decoded = keras.layers.Conv2D(1, (3, 3), activation='linear', padding='same')(x)

In [45]:
autoencoder = keras.Model(input_layer, decoded)

In [46]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1, 16000)          0         
_________________________________________________________________
melspectrogram_5 (Melspectro (None, 63, 63, 1)         279359    
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 63, 63, 128)       1280      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 32, 32, 64)        73792     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 16, 16, 64)        36928     
__________

In [ ]:
def mel_loss(y_true, mel_pred):
    mel_true

In [47]:
autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [48]:
(x,y,cw) = next(ex_generator(batch_size=1000, state="submission"))

In [50]:
x.shape

(1000, 1, 16000)

In [54]:
autoencoder.fit(x=x,y=y,batch_size=32)

ValueError: Error when checking target: expected conv2d_66 to have 4 dimensions, but got array with shape (1000, 1)